<a href="https://colab.research.google.com/github/AlexeyTri/Home_work_NN/blob/main/HomeWork_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
import scipy
from scipy.sparse import coo_matrix, hstack

In [2]:
df_train = pd.read_csv("/content/sample_data/salary-train.csv")
df_test = pd.read_csv("https://raw.githubusercontent.com/AlexeyTri/Home_work_NN/refs/heads/main/data/salary-test-mini.csv")

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   FullDescription     60000 non-null  object
 1   LocationNormalized  60000 non-null  object
 2   ContractTime        44418 non-null  object
 3   SalaryNormalized    60000 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.8+ MB


In [4]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   FullDescription     2 non-null      object 
 1   LocationNormalized  2 non-null      object 
 2   ContractTime        2 non-null      object 
 3   SalaryNormalized    0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 196.0+ bytes


In [5]:
df_train['FullDescription'] = df_train['FullDescription'].str.lower().str.replace('[^a-zA-Z0-9]', " ", regex=True)
df_train['LocationNormalized'] = df_train['LocationNormalized'].str.lower().str.replace('[^a-zA-Z0-9]', " ", regex=True)
df_train['ContractTime'] = df_train['ContractTime'].str.lower().str.replace('[^a-zA-Z0-9]', " ", regex=True)

In [6]:
df_test['FullDescription'] = df_test['FullDescription'].str.lower().str.replace('[^a-zA-Z0-9]', " ", regex=True)
df_test['LocationNormalized'] = df_test['LocationNormalized'].str.lower().str.replace('[^a-zA-Z0-9]', " ", regex=True)
df_test['ContractTime'] = df_test['ContractTime'].str.lower().str.replace('[^a-zA-Z0-9]', " ", regex=True)

In [7]:
tfidf = TfidfVectorizer(min_df=5)
tfidf.fit_transform(df_train['FullDescription'].values)
tfidf_matrix = tfidf.fit_transform(df_train['FullDescription'].values)

In [8]:
tfidf_matrix_test = tfidf.transform(df_test['FullDescription'].values)
tfidf_matrix_test_dataframe = pd.DataFrame(tfidf_matrix_test.toarray(), columns=tfidf.get_feature_names_out())

In [9]:
tfidf_matrix_dataframe = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

In [10]:
tfidf_matrix_test_dataframe.shape[1] == tfidf_matrix_dataframe.shape[1]

True

In [11]:
df_train['ContractTime'].fillna('nan', inplace=True)
df_train['LocationNormalized'].fillna('nan', inplace=True)

/tmp/ipython-input-3712495894.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['ContractTime'].fillna('nan', inplace=True)
/tmp/ipython-input-3712495894.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [12]:
df_test['ContractTime'].fillna('nan', inplace=True)
df_test['LocationNormalized'].fillna('nan', inplace=True)

/tmp/ipython-input-4105289674.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_test['ContractTime'].fillna('nan', inplace=True)
/tmp/ipython-input-4105289674.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [13]:
contract_time = pd.get_dummies(df_train['ContractTime'], prefix='contTime', drop_first=False, dtype=float)
loc_normal = pd.get_dummies(df_train['LocationNormalized'], prefix='locnorm', drop_first=False, dtype=float)

In [14]:
contract_time_test = pd.get_dummies(df_test['ContractTime'], prefix='contTime', drop_first=False, dtype=float)
loc_normal_test = pd.get_dummies(df_test['LocationNormalized'], prefix='locnorm', drop_first=False, dtype=float)

In [15]:
contract_time.shape, contract_time_test.shape, loc_normal.shape, loc_normal_test.shape

((60000, 3), (2, 2), (60000, 1763), (2, 2))

In [16]:
tfidf_matrix_dataframe_coo_matrix = coo_matrix(tfidf_matrix_dataframe.values)
contract_time_coo_matrix = coo_matrix(contract_time.values)
loc_normal_coo_matrix = coo_matrix(loc_normal.values)

In [17]:
tfidf_matrix_test_dataframe_coo_matrix = coo_matrix(tfidf_matrix_test_dataframe.values)
contract_time_test_coo_matrix = coo_matrix(contract_time_test.values)
loc_normal_test_coo_matrix = coo_matrix(loc_normal_test.values)

In [18]:
data_train = scipy.sparse.hstack([tfidf_matrix_dataframe_coo_matrix, contract_time_coo_matrix, loc_normal_coo_matrix])

In [19]:
data_test = scipy.sparse.hstack([tfidf_matrix_test_dataframe_coo_matrix, contract_time_test_coo_matrix, loc_normal_test_coo_matrix])

In [20]:
y = df_train['SalaryNormalized']

In [21]:
data_train.shape[1] - data_test.shape[1]

1762

In [22]:
ridge = Ridge(alpha=1)
ridge.fit(data_train, y)

Ridge(alpha=1)

In [23]:
data_test_new = np.hstack([data_test.toarray(), np.zeros((2, 1762))])
data_test_new.shape

(2, 24627)

In [24]:
ridge.predict(data_test_new)

array([55733.59501765, 36477.24051463])